In [1]:
import pandas as pd
import numpy as np
from util_funcs import indexing_1
from eval_metrics_foos import *

In [2]:
df = pd.read_excel("ing_data_1.xlsx")
df.head()

,id,app,source,title,review,rating,user,date,createdate,deep_translator,...,question,complaint,statement,praise,user interface,design,finance,credit,customer relationship,performance
0,35,ING,google,NaN,Çok Kullanışlı ve anlaşılır bir uygulama ve mü...,5,Vural öz,2023-12-11,2023-12-14,It is a very useful and understandable applica...,...,0.007078,0.000804,0.628283,0.984809,0.879359,0.001819,0.448141,0.335250,0.979935,0.204437
1,40,ING,google,NaN,12.10.2023 de 5 adet otomatik ödeme talimatı v...,1,Mevlut Yilmaz,2023-12-12,2023-12-14,I gave 5 automatic payment orders on 12.10.202...,...,0.972236,0.927619,0.666440,0.000897,0.813887,0.008926,0.995085,0.701604,0.740115,0.398567
2,36,ING,google,NaN,"Gayet başarılı, kampanyalar artarsa ve kampany...",5,Atakan TANRIVER,2023-12-11,2023-12-14,"It is very successful, it would be good if the...",...,0.503769,0.001884,0.395456,0.980226,0.812882,0.180230,0.184196,0.524749,0.574716,0.987284
3,14,ING,google,NaN,ING MOBİL Uygulaması çok ama çok kötü Kullanım...,1,Mehmet Gungor,2023-11-28,2023-12-14,"ING MOBILE Application is very, very bad. It i...",...,0.676765,0.992421,0.696690,0.000450,0.948215,0.127819,0.936973,0.666830,0.530031,0.960343
4,79,ING,google,NaN,Keşke uygulamayı güncellemeseydiniz kullanamıy...,1,Gamze Arıcan,2023-10-12,2023-12-14,"I wish you didn't update the application, I ca...",...,0.723204,0.995303,0.925061,0.036337,0.853893,0.055612,0.499166,0.373095,0.423109,0.116472


In [3]:
sentiments = pd.read_excel("sentiments__.xlsx")
sentiments.head()

,id,review,Review Person,USER_RATING
0,6544,En iyi banka uygulaması her işlem en hızlı ing...,OY,5
1,10932,Siz %30 kazanın vadelide müşterinize %7verin n...,OY,1
2,700,"Her şeyi iyi,hoş ve güzel bir banka fakat, Tur...",OY,4
3,2349,Nasıl daha kullanışsız ve kötü yaparız uygulam...,OY,1
4,25521,Günlük para çekim limitim 1.000 tl. bozdur boz...,OY,2


In [4]:
df = indexing_1(sentiments, df, "id", "Review Person", "Reviewer")
df = indexing_1(sentiments, df, "id", "USER_RATING", "USER_RATING")

In [99]:
df_cleaned = df.dropna(subset=["USER_RATING"])
df_cleaned.reset_index(inplace=True, drop=True)

df_cleaned["USER_RATING"] = df_cleaned["USER_RATING"].astype(int)

/var/folders/9d/xym4108s4255j4wn5n1n7n3c0000gn/T/ipykernel_18166/273911296.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["USER_RATING"] = df_cleaned["USER_RATING"].astype(int)


In [100]:
for i in range(1,5):
    df_cleaned[f"c_score_{i}_rounded"] = (df_cleaned[f"c_score_{i}"] / 2).round().astype(int)
    df_cleaned[f"c_score_{i}_rounded"] = df_cleaned[f"c_score_{i}_rounded"].replace(0,1)

df_cleaned[f"Sentiment_Mean_Score_rounded"] = (df_cleaned[f"Sentiment_Mean_Score"] / 2).round().astype(int)
df_cleaned[f"Sentiment_Mean_Score_rounded"] = df_cleaned[f"Sentiment_Mean_Score_rounded"].replace(0,1)

/var/folders/9d/xym4108s4255j4wn5n1n7n3c0000gn/T/ipykernel_18166/2045642467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[f"c_score_{i}_rounded"] = (df_cleaned[f"c_score_{i}"] / 2).round().astype(int)
/var/folders/9d/xym4108s4255j4wn5n1n7n3c0000gn/T/ipykernel_18166/2045642467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[f"c_score_{i}_rounded"] = df_cleaned[f"c_score_{i}_rounded"].replace(0,1)
/var/folders/9d/xym4108s4255j4wn5n1n7n3c0000gn/T/ipykernel_18166/2045642467.py:2:

In [101]:
#df_cleaned.to_excel("ing_sentiments_cleaned.xlsx",index=False)

In [102]:
y_true = df_cleaned["USER_RATING"]

In [103]:
def classification_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)

    accuracy = accuracy_score(y_true, y_pred)

    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    f1_score_macro = f1_score(y_true, y_pred, average='macro')

    precision_micro = precision_score(y_true, y_pred, average='micro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    f1_score_micro = f1_score(y_true, y_pred, average='micro')

    print("confusion: \n", conf_matrix)
    print("accuracy: ", accuracy)
    print("precision_macro: ", precision_macro)
    print("recall_macro: ", recall_macro)
    print("f1_score_macro: ", f1_score_macro)
    print("precision_micro: ", precision_micro)
    print("recall_micro: ", recall_micro)
    print("f1_score_micro: ", f1_score_micro)

In [104]:
classification_metrics(y_true, df_cleaned["c_score_1_rounded"])

confusion: 
 [[33  0  1 10  9]
 [55  4  4  8  5]
 [39  5  8  7  1]
 [30  2  6 16  3]
 [36  1  2  3 12]]
accuracy:  0.24333333333333335
precision_macro:  0.3297813067761254
recall_macro:  0.2623060796645702
f1_score_macro:  0.2318557213832107
precision_micro:  0.24333333333333335
recall_micro:  0.24333333333333335
f1_score_micro:  0.24333333333333335


In [105]:
classification_metrics(y_true, df_cleaned["c_score_2_rounded"])

confusion: 
 [[36  0  0  0 17]
 [42  0  0  0 34]
 [38  0  0  0 22]
 [12  0  0  0 45]
 [ 5  0  0  0 49]]
accuracy:  0.2833333333333333
precision_macro:  0.11281797307640358
recall_macro:  0.317330538085255
f1_score_macro:  0.16610713764413954
precision_micro:  0.2833333333333333
recall_micro:  0.2833333333333333
f1_score_micro:  0.2833333333333333


/Users/okanyenigun/Desktop/codes/projects/genai_work/notebooks/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [106]:
classification_metrics(y_true, df_cleaned["c_score_3_rounded"])

confusion: 
 [[31  0  1  8 13]
 [40  0  1  5 30]
 [33  0  1  5 21]
 [48  0  0  2  7]
 [42  0  0  1 11]]
accuracy:  0.15
precision_macro:  0.14450231689356657
recall_macro:  0.16807275000919492
f1_score_macro:  0.09916098693188476
precision_micro:  0.15
recall_micro:  0.15
f1_score_micro:  0.15


/Users/okanyenigun/Desktop/codes/projects/genai_work/notebooks/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [107]:
classification_metrics(y_true, df_cleaned["c_score_4_rounded"])

confusion: 
 [[17  1  0 15 20]
 [12  0  2 13 49]
 [ 3  0  0  6 51]
 [ 4  0  0  9 44]
 [ 7  0  0 11 36]]
accuracy:  0.20666666666666667
precision_macro:  0.1484031007751938
recall_macro:  0.2290632240979808
f1_score_macro:  0.1599586791515925
precision_micro:  0.20666666666666667
recall_micro:  0.20666666666666667
f1_score_micro:  0.20666666666666667


In [108]:
classification_metrics(y_true, df_cleaned["Sentiment_Mean_Score_rounded"])

confusion: 
 [[12 30 11  0  0]
 [ 3 41 29  3  0]
 [ 4 25 29  2  0]
 [ 0 12 33 12  0]
 [ 1 10 27 16  0]]
accuracy:  0.31333333333333335
precision_macro:  0.3071800384610791
recall_macro:  0.2919496855345912
f1_score_macro:  0.26499850184075574
precision_micro:  0.31333333333333335
recall_micro:  0.31333333333333335
f1_score_micro:  0.31333333333333335


/Users/okanyenigun/Desktop/codes/projects/genai_work/notebooks/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
classification_metrics(y_true, df_cleaned["rating"])

confusion: 
 [[41  3  3  1  5]
 [33  6 14 12 11]
 [13  6  6 13 22]
 [ 1  0  0 13 43]
 [ 0  0  0  2 52]]
accuracy:  0.3933333333333333
precision_macro:  0.36696585409344246
recall_macro:  0.42871308249659784
f1_score_macro:  0.33589251677567755
precision_micro:  0.3933333333333333
recall_micro:  0.3933333333333333
f1_score_micro:  0.3933333333333333


In [ ]:
# call center summarization
# call center müşteriye ilk ne diyelim



In [110]:
df_cleaned['USER_RATING_binary'] = np.where(df_cleaned['USER_RATING'] > 2.5, 1, 0)
df_cleaned['c_score_1_rounded_binary'] = np.where(df_cleaned['c_score_1_rounded'] > 2.5, 1, 0)
df_cleaned['c_score_2_rounded_binary'] = np.where(df_cleaned['c_score_2_rounded'] > 2.5, 1, 0)
df_cleaned['c_score_3_rounded_binary'] = np.where(df_cleaned['c_score_3_rounded'] > 2.5, 1, 0)
df_cleaned['c_score_4_rounded_binary'] = np.where(df_cleaned['c_score_4_rounded'] > 2.5, 1, 0)
df_cleaned['Sentiment_Mean_Score_rounded_binary'] = np.where(df_cleaned['Sentiment_Mean_Score_rounded'] > 2.5, 1, 0)
df_cleaned['rating_binary'] = np.where(df_cleaned['rating'] > 2.5, 1, 0)


/var/folders/9d/xym4108s4255j4wn5n1n7n3c0000gn/T/ipykernel_18166/4055661369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['USER_RATING_binary'] = np.where(df_cleaned['USER_RATING'] > 2.5, 1, 0)
/var/folders/9d/xym4108s4255j4wn5n1n7n3c0000gn/T/ipykernel_18166/4055661369.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['c_score_1_rounded_binary'] = np.where(df_cleaned['c_score_1_rounded'] > 2.5, 1, 0)
/var/folders/9d/xym4108s4255j4wn5n1n7n3c0000gn/T/ipykernel_18166/4055661369.py

In [111]:
y_true = df_cleaned["USER_RATING_binary"]

In [112]:
classification_metrics(y_true, df_cleaned["c_score_1_rounded_binary"])

confusion: 
 [[ 92  37]
 [113  58]]
accuracy:  0.5
precision_macro:  0.5296534017971759
recall_macro:  0.5261797905616755
f1_score_macro:  0.49349421457836207
precision_micro:  0.5
recall_micro:  0.5
f1_score_micro:  0.5


In [113]:
classification_metrics(y_true, df_cleaned["c_score_2_rounded_binary"])

confusion: 
 [[ 78  51]
 [ 55 116]]
accuracy:  0.6466666666666666
precision_macro:  0.6405384719283238
recall_macro:  0.6415068679450564
f1_score_macro:  0.6409051899363115
precision_micro:  0.6466666666666666
recall_micro:  0.6466666666666666
f1_score_micro:  0.6466666666666666


In [114]:
classification_metrics(y_true, df_cleaned["c_score_3_rounded_binary"])

confusion: 
 [[ 71  58]
 [123  48]]
accuracy:  0.39666666666666667
precision_macro:  0.4094047850612721
recall_macro:  0.41554467564259484
f1_score_macro:  0.3930994400420248
precision_micro:  0.39666666666666667
recall_micro:  0.39666666666666667
f1_score_micro:  0.39666666666666667


In [115]:
classification_metrics(y_true, df_cleaned["c_score_4_rounded_binary"])

confusion: 
 [[ 30  99]
 [ 14 157]]
accuracy:  0.6233333333333333
precision_macro:  0.6475497159090908
recall_macro:  0.575343397252822
f1_score_macro:  0.5410919034533173
precision_micro:  0.6233333333333333
recall_micro:  0.6233333333333333
f1_score_micro:  0.6233333333333333


In [116]:
classification_metrics(y_true, df_cleaned["Sentiment_Mean_Score_rounded_binary"])

confusion: 
 [[ 86  43]
 [ 52 119]]
accuracy:  0.6833333333333333
precision_macro:  0.6788781535158347
recall_macro:  0.6812865497076024
f1_score_macro:  0.6794547356345109
precision_micro:  0.6833333333333333
recall_micro:  0.6833333333333333
f1_score_micro:  0.6833333333333333


In [117]:
classification_metrics(y_true, df_cleaned["rating_binary"])

confusion: 
 [[ 83  46]
 [ 20 151]]
accuracy:  0.78
precision_macro:  0.7861613523236903
recall_macro:  0.7632258941928465
f1_score_macro:  0.7680847076461769
precision_micro:  0.78
recall_micro:  0.78
f1_score_micro:  0.78
